# Risk calculator process

## Importing modules for running process
In this section all modules which are used by the system are loaded in memory

In [1]:
# Import libraries
import os
import glob
import pandas as pd

# Import modules
import deforestation as defo
import administrative_level as adle
import livestock_plot as plot
import risk as risk
import mobilization as mobi
import importer as impo
print("Done!")

Done!


## Setting global parameters
In this section all configurations about folders and files to set up
the system are created. These configurations are global variables for the system

In [2]:
# Global folders
#root_folder = "/home/jovyan/work/BID/dci_fauno/"
root_folder = "D:\\CIAT\\Code\\BID\\dci_fauno"
data_folder = os.path.join(root_folder, "data","risk_calculator")
inputs = os.path.join(data_folder,"inputs")
outputs = os.path.join(data_folder,"outputs")
conf_folder = os.path.join(data_folder,"conf")
# Deforestation folders
def_folder = os.path.join(inputs,"def")
# Administrative level folders
adm_folder = os.path.join(inputs,"adm")
# Livestock plots
plot_folder = os.path.join(inputs,"plot")
# Mobilization
mob_folder = os.path.join(inputs,"mob")

# Parameters file
conf_xls = pd.ExcelFile(os.path.join(conf_folder,"conf.xlsx"))
conf = conf_xls.parse("conf")
buffer = conf_xls.parse("buffer")

# Global parameters from conf.xlsx file
glo_crs = int(conf.loc[conf["parameter"] == "glo_crs","value"].values[0])
glo_crs_wgs84 = int(conf.loc[conf["parameter"] == "glo_crs_wgs84","value"].values[0])

# Creating folders
# Create folder for deforestation files
if not os.path.exists(def_folder):    
    os.mkdir(def_folder)
# Create folder for administrative level files
if not os.path.exists(adm_folder):    
    os.mkdir(adm_folder)
# Create folder for livestock plot files
if not os.path.exists(plot_folder):    
    os.mkdir(plot_folder)
# Create folder for mobilization files
if not os.path.exists(mob_folder):    
    os.mkdir(mob_folder)

print("Done!")

Done!


## Downloading data
This section downloads all data require by the system. It keeps all raw files, it will allow to debug if it would be necessary

### Administrative level - Download and unzip
In this section all data about administrative lever of the country will be downloaded and extracted
into specific folders.

In [ ]:
# Getting parameters for downloading adiministrative sources
adm_url_source = conf.loc[conf["parameter"] == "adm_url_source","value"].values[0]
# Download files from source in zip format and extract them
adle.download_data(adm_folder, adm_url_source)
print("Done!")

### Deforestation - Download and unzip
In this section all data about deforestation for a country are downloaded and extracted
into specific folders.

In [ ]:
# Getting parameters for downloading deforestation files
url_base = conf.loc[conf["parameter"] == "def_src_base_url","value"].values[0]
file_name_base = conf.loc[conf["parameter"] == "def_src_file_name","value"].values[0]
def_periods = conf.loc[conf["parameter"] == "def_periods","value"].values[0].split(",")

# Download files from source in zip format and extract them
defo.download_deforestation(def_folder, url_base, file_name_base, def_periods)
print("Done!")

## Fixing data
This chaptar fix the data downloaded into files and formats, in order to can process information

### Administrative level - Reproject
This process reprojects to CRS destiny the original database

In [ ]:
adm_src_adm1_name = conf.loc[conf["parameter"] == "adm_src_adm1_name","value"].values[0]
adm_src_adm1_id = conf.loc[conf["parameter"] == "adm_src_adm1_id","value"].values[0]
adm_src_adm2_name = conf.loc[conf["parameter"] == "adm_src_adm2_name","value"].values[0]
adm_src_adm2_id = conf.loc[conf["parameter"] == "adm_src_adm2_id","value"].values[0]
adm_src_adm3_name = conf.loc[conf["parameter"] == "adm_src_adm3_name","value"].values[0]
adm_src_adm3_id = conf.loc[conf["parameter"] == "adm_src_adm3_id","value"].values[0]
adm_src_area = conf.loc[conf["parameter"] == "adm_src_area","value"].values[0]
adm_src_geometry = conf.loc[conf["parameter"] == "adm_src_geometry","value"].values[0]

# Reporject file
adle.reproject_source(adm_folder, glo_crs,
                      adm_src_adm1_name, adm_src_adm1_id, 
                      adm_src_adm2_name, adm_src_adm2_id, 
                      adm_src_adm3_name, adm_src_adm3_id, 
                      adm_src_area, adm_src_geometry)
print("Done!")

### Deforestation - Extract deforestation values
This section takes the rasters files, which were downloaded, then extract just deforestation pixels
and create a new raster with this data. Finally it creates a summary raster which merge all
others raster files in just one. The output raster files are standarized, it means all pixels are in 
the same size, the CRS is the same for all, and are clipped with the same size

In [3]:
# Getting parameters for filtering deforestation pixels
def_value = int(conf.loc[conf["parameter"] == "def_value","value"].values[0])
def_pixel_size = float(conf.loc[conf["parameter"] == "def_pixel_size","value"].values[0])

print("Extracting!")
defo.extract_deforestation(def_folder, def_value, glo_crs, def_pixel_size)
print("Summaryzing!")
defo.summary_deforestation(def_folder)
print("Done!")

Extracting!
Calculating parameters for new files
file: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\content\Cambio_Cobertura_Bosque_No_Bosque_2010_2012\Geotiff\DCCB_SMBYC_CBBQ_V5_2010_2012.tif
file: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\content\Cambio_Cobertura_Bosque_No_Bosque_2012_2013\Geotiff\DCCB_SMBYC_CBBQ_V5_2012_2013.tif
file: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\content\Cambio_Cobertura_Bosque_No_Bosque_2013_2014\Geotiff\DCCB_SMBYC_CBBQ_V6_2013_2014.tif
file: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\content\Cambio_Cobertura_Bosque_No_Bosque_2014_2015\GeoTiff\DCCB_SMBYC_CBBQ_V7_2014_2015.tif
file: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\content\Cambio_Cobertura_Bosque_No_Bosque_2015_2016\Geotiff\DCCB_SMBYC_CBBQ_V7_2015_2016.tif
file: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\content\Cambio_Cobertura_Bosque_No_Bosque_2016_2017\TTTHM4RG3_BEA_T_reprojected_2016_2017.tif
fi

d:\ciat\virtualenv\jupyter\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Opening: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\DCCB_SMBYC_CBBQ_V5_2010_2012_tmp.tif
Cropping raster
Dimention: H=61079 W=45525
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\raster_detail\2010.tif
Deleting tmp: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\DCCB_SMBYC_CBBQ_V5_2010_2012_tmp.tif
Working: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\content\Cambio_Cobertura_Bosque_No_Bosque_2012_2013\Geotiff\DCCB_SMBYC_CBBQ_V5_2012_2013.tif
Reprojecting: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\DCCB_SMBYC_CBBQ_V5_2012_2013_tmp.tif
Opening: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\DCCB_SMBYC_CBBQ_V5_2012_2013_tmp.tif
Cropping raster
Dimention: H=61079 W=45525
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\raster_detail\2012.tif
Deleting tmp: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\DCCB_SMBYC_CBBQ_V5_2012_2013

### Deforestation - Parsing to shapefile
In this section the raster files, which were located into rasters folders, are parsed to shapefile format.
Both folders are transformed detail and summary

In [3]:
# Parsing to shapefile
defo.to_shp(def_folder)
print("Done!")

Opening: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\raster_detail\2010.tif
Transforming
Creating shapefile crs: {'init': 'epsg:3116'}


d:\ciat\virtualenv\jupyter\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


            lon           lat                         geometry
0  1.284164e+06  1.850139e+06  POINT (1284163.792 1850139.356)
1  1.284195e+06  1.850139e+06  POINT (1284194.509 1850139.356)
2  1.284133e+06  1.850109e+06  POINT (1284133.075 1850108.639)
3  1.284164e+06  1.850109e+06  POINT (1284163.792 1850108.639)
4  1.284195e+06  1.850109e+06  POINT (1284194.509 1850108.639)
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\shp_detail\2010\shapefile.shp
Opening: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\raster_detail\2012.tif
Transforming
Creating shapefile crs: {'init': 'epsg:3116'}


d:\ciat\virtualenv\jupyter\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


            lon           lat                         geometry
0  1.304345e+06  1.856529e+06  POINT (1304344.964 1856528.595)
1  1.304376e+06  1.856529e+06  POINT (1304375.681 1856528.595)
2  1.304314e+06  1.856498e+06  POINT (1304314.247 1856497.878)
3  1.304345e+06  1.856498e+06  POINT (1304344.964 1856497.878)
4  1.304376e+06  1.856498e+06  POINT (1304375.681 1856497.878)
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\shp_detail\2012\shapefile.shp
Opening: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\raster_detail\2013.tif
Transforming
Creating shapefile crs: {'init': 'epsg:3116'}


...

            lon           lat                         geometry
0  1.304345e+06  1.856528e+06  POINT (1304344.853 1856528.489)
1  1.304376e+06  1.856528e+06  POINT (1304375.570 1856528.489)
2  1.304314e+06  1.856498e+06  POINT (1304314.136 1856497.772)
3  1.304345e+06  1.856498e+06  POINT (1304344.853 1856497.772)
4  1.304376e+06  1.856498e+06  POINT (1304375.570 1856497.772)
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\shp_summary\2016\shapefile.shp
Opening: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\raster_summary\2017.tif
Transforming
Creating shapefile crs: {'init': 'epsg:3116'}


d:\ciat\virtualenv\jupyter\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


            lon           lat                         geometry
0  1.304345e+06  1.856528e+06  POINT (1304344.853 1856528.489)
1  1.304376e+06  1.856528e+06  POINT (1304375.570 1856528.489)
2  1.304314e+06  1.856498e+06  POINT (1304314.136 1856497.772)
3  1.304345e+06  1.856498e+06  POINT (1304344.853 1856497.772)
4  1.304376e+06  1.856498e+06  POINT (1304375.570 1856497.772)
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\shp_summary\2017\shapefile.shp
Opening: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\raster_summary\2018.tif
Transforming
Creating shapefile crs: {'init': 'epsg:3116'}


d:\ciat\virtualenv\jupyter\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


            lon           lat                         geometry
0  1.304345e+06  1.856528e+06  POINT (1304344.853 1856528.489)
1  1.304376e+06  1.856528e+06  POINT (1304375.570 1856528.489)
2  1.304314e+06  1.856498e+06  POINT (1304314.136 1856497.772)
3  1.304345e+06  1.856498e+06  POINT (1304344.853 1856497.772)
4  1.304376e+06  1.856498e+06  POINT (1304375.570 1856497.772)
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\shp_summary\2018\shapefile.shp
Done!


### Plots
This section compiles all CSV files with information about plots for all years. It clear all records, 
validating that all plots have coordinates and these coordinates are good. Further it sets the region
and locality according to shapefile of reference. Finally it creates a new layer with all data compiled

In [3]:
# Getting parameters for processing data
plo_character_file = conf.loc[conf["parameter"] == "plo_character_file","value"].values[0]
plo_src_ext_id = conf.loc[conf["parameter"] == "plo_src_ext_id","value"].values[0]
plo_src_lat = conf.loc[conf["parameter"] == "plo_src_lat","value"].values[0]
plo_src_lon = conf.loc[conf["parameter"] == "plo_src_lon","value"].values[0]
plo_src_animals = conf.loc[conf["parameter"] == "plo_src_animals","value"].values[0]
plo_src_adm1 = conf.loc[conf["parameter"] == "plo_src_adm1","value"].values[0]
plo_src_adm2 = conf.loc[conf["parameter"] == "plo_src_adm2","value"].values[0]
plo_src_adm3 = conf.loc[conf["parameter"] == "plo_src_adm3","value"].values[0]
plo_lim_lat_low =  float(conf.loc[conf["parameter"] == "plo_lim_lat_low","value"].values[0])
plo_lim_lat_upp =  float(conf.loc[conf["parameter"] == "plo_lim_lat_upp","value"].values[0])
plo_lim_lon_low =  float(conf.loc[conf["parameter"] == "plo_lim_lon_low","value"].values[0])
plo_lim_lon_upp =  float(conf.loc[conf["parameter"] == "plo_lim_lon_upp","value"].values[0])
# Getting parameters for creating shp
plo_src_crs = int(conf.loc[conf["parameter"] == "plo_src_crs","value"].values[0])

has_adm = (plo_src_adm1 != "") & (plo_src_adm2 != "") & (plo_src_adm3 != "")

# Compiling data
plot.processing_raw_data(plot_folder, plo_character_file, 
              plo_src_ext_id, plo_src_lat, plo_src_lon, plo_src_animals,
              plo_lim_lat_low, plo_lim_lat_upp, plo_lim_lon_low, plo_lim_lon_upp,
              has_adm, plo_src_adm1, plo_src_adm2, plo_src_adm3)
# Fixing data
path_adm_shp = os.path.join(adm_folder,"fixed","adm.shp")
plot.create_data(plot_folder, path_adm_shp, plo_src_crs, glo_crs)

print("Done!")

Reading: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\plot\download\2019.csv


d:\ciat\virtualenv\jupyter\lib\site-packages\IPython\core\interactiveshell.py:3347: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Fixing columns
Cleaning records
     ext_id       lat        lon  animals  year       adm1             adm2  \
0   1589850 -4.006928 -70.134470        1  2019   AMAZONAS          LETICIA   
1   1589861 -4.176944 -69.943611       28  2019   AMAZONAS          LETICIA   
2   1589877 -4.141955 -70.010603       35  2019   AMAZONAS          LETICIA   
8   1589914 -3.771690 -70.359310        4  2019   AMAZONAS  PUERTO NARIï¿½O   
10  1510795  5.860740 -75.501670       17  2019  ANTIOQUIA        ABEJORRAL   

               adm3  
0           LETICIA  
1           LETICIA  
2           LETICIA  
8   PUERTO NARIï¿½O  
10      ALTO BONITO  
Logging: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\plot\log\err-data-bad_plots.csv
Goruping and filtering max
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\plot\content\plots.csv
Opening: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\plot\content\plots.csv
Transforming to points


d:\ciat\virtualenv\jupyter\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Reprojecting
Opening shp reference: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\adm\fixed\adm.shp
Joining spatial
       ext_id       lat        lon  animals  year      adm1  \
0      400165  5.282530 -71.963700     7470  2019  CASANARE   
1      400292  5.342888 -71.967508     6673  2019  CASANARE   
662   1189289  5.220117 -71.931157     1398  2019  CASANARE   
1092  1475236  5.288035 -71.963668     1115  2019  CASANARE   
3030  1259268  5.229343 -71.942858      676  2019  CASANARE   

                      adm2       adm3  animals_max  \
0     SAN LUIS DE PALENQUE  ALGODONAL         7470   
1     SAN LUIS DE PALENQUE   JAGUEYES         6673   
662   SAN LUIS DE PALENQUE   JAGUEYES         1398   
1092  SAN LUIS DE PALENQUE   JAGUEYES         1115   
3030  SAN LUIS DE PALENQUE   JAGUEYES          676   

                             geometry  index_right adm1_id adm1_name adm2_id  \
0     POINT (1234367.895 1076294.221)         4243      85  CASANARE   85325   
1     POINT

### Calculating buffer by plot
In this section, the script creates a buffer for all plots. It uses the following ecuation to set radious for the buffer: amount of animals in plot / field capacity. 

In [4]:
# Creatting buffer
plot.create_buffer(plot_folder, buffer, glo_crs)
print("Done!")

Opening plots shape file: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\plot\fixed\plots\plots.shp


d:\ciat\virtualenv\jupyter\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Mergin with field capacity
Creating buffer
Reprojecting
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\plot\fixed\buffer\buffer.shp
Done!


## Mobilization

### Fixing mobilization data

In [3]:
# Getting parameters for processing data
mob_origin_name = conf.loc[conf["parameter"] == "mob_origin_name","value"].values[0]
mob_destination_name = conf.loc[conf["parameter"] == "mob_destination_name","value"].values[0]
mob_type_name = conf.loc[conf["parameter"] == "mob_type_name","value"].values[0]
mob_columns_exclude = map(str.strip, conf.loc[conf["parameter"] == "mob_columns_exclude","value"].values[0].split(","))

mobi.fixed_data(mob_folder,mob_columns_exclude,mob_origin_name,mob_destination_name,mob_type_name)
print("Done!")

Processing: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\mob\content\2017.csv
Removing columns
Pivoting data by year
Summarizing mobilization
Fixing columns names
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\mob\fixed\2017.csv
Processing: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\mob\content\2018.csv
Removing columns
Pivoting data by year
Summarizing mobilization
Fixing columns names
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\mob\fixed\2018.csv
Done!


## Analyzing plots

### Calculating deforested areas by plot
In this section the system calculates the proportion of deforested areas by plot. It combines raster deforestation files
and buffer shapefile.

In [10]:
# Getting parameters for processing data
ris_years = map(str.strip, conf.loc[conf["parameter"] == "ris_years","value"].values[0].split(","))
ris_types_analysis =  map(str.strip, conf.loc[conf["parameter"] == "ris_types_analysis","value"].values[0].split(","))
def_pixel_size = float(conf.loc[conf["parameter"] == "def_pixel_size","value"].values[0])

# De
#risk.deforestation_plot(inputs, ris_years, ris_types_analysis, def_pixel_size)
risk.deforestation_plot(inputs, [2010,2012,2013,2014,2015,2016,2017,2018], ["summary"], def_pixel_size)

Opening shp: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\plot\fixed\buffer\buffer.shp
Type of analysis: summary
Processing: 2010
Opening raster: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\raster_summary\2010.tif
Calculating zonal stats


RasterioIOError: Read or write failed. D:/CIAT/Code/BID/dci_fauno/data/risk_calculator/inputs/def/fixed/raster_summary/2010.tif, band 1: IReadBlock failed at X offset 0, Y offset 26771: _TIFFFetchStrileValue:Cannot allocate strip offset and bytecount arrays

### Calculating distance between plots and near deforestation

In [3]:
# Getting parameters for processing data
ris_years = map(str.strip, conf.loc[conf["parameter"] == "ris_years","value"].values[0].split(","))
ris_types_analysis =  map(str.strip, conf.loc[conf["parameter"] == "ris_types_analysis","value"].values[0].split(","))

risk.distance_plot(inputs, ris_years, ris_types_analysis)
print("Done!")

Processing type: detail
Processing year: 2017
Opening areas shp: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\plot\fixed\def\detail\2017\areas.shp
Opening deforestation shp: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\shp_detail\2017\shapefile.shp
Fixing fields
Starting process parallel
Joining results
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\plot\fixed\dis\detail\2017\areas_distance.shp
Processing year: 2018
Opening areas shp: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\plot\fixed\def\detail\2018\areas.shp
Opening deforestation shp: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\shp_detail\2018\shapefile.shp
Fixing fields
Starting process parallel
Joining results
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\plot\fixed\dis\detail\2018\areas_distance.shp
Processing type: summary
Done!


### Calculating direct risk
It changes teh CSR to WGS84 for all new layers with risk

In [3]:
# Getting parameters for processing data
ris_years = map(str.strip, conf.loc[conf["parameter"] == "ris_years","value"].values[0].split(","))
ris_types_analysis =  map(str.strip, conf.loc[conf["parameter"] == "ris_types_analysis","value"].values[0].split(","))

risk.risk_direct(inputs, ris_years, ris_types_analysis, glo_crs_wgs84)
print("Done!")

Processing type: detail
Processing year: 2010
Opening areas shp: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\plot\fixed\dis\detail\2010\areas_distance.shp
Reprojecting to: {'init': 'epsg:4326', 'no_defs': True}


d:\ciat\virtualenv\jupyter\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Starting process parallel


d:\ciat\virtualenv\jupyter\lib\site-packages\pyproj\crs\crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)


Joining results
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\plot\fixed\risk\detail\2010\rd.shp
Processing year: 2012
Opening areas shp: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\plot\fixed\dis\detail\2012\areas_distance.shp
Reprojecting to: {'init': 'epsg:4326', 'no_defs': True}
Starting process parallel
Joining results
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\plot\fixed\risk\detail\2012\rd.shp
Processing year: 2013
Opening areas shp: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\plot\fixed\dis\detail\2013\areas_distance.shp
Reprojecting to: {'init': 'epsg:4326', 'no_defs': True}
Starting process parallel
Joining results
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\plot\fixed\risk\detail\2013\rd.shp
Processing year: 2014
Opening areas shp: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\plot\fixed\dis\detail\2014\areas_distance.shp
Reprojecting to: {'init': 'epsg:4326', 'no_defs': True}
Starting pro

### Calculating total risk for each plot
It takes into account all paramters: direct risk, risk of all in plots and also 

In [ ]:
# Getting parameters for processing data
ris_years = map(str.strip, conf.loc[conf["parameter"] == "ris_years","value"].values[0].split(","))
ris_types_analysis =  map(str.strip, conf.loc[conf["parameter"] == "ris_types_analysis","value"].values[0].split(","))
ris_type_plot = conf.loc[conf["parameter"] == "ris_type_plot","value"].values[0]

#risk.total_risk(inputs, ris_years, ris_types_analysis)
risk.total_risk(inputs, ["2017","2018"], ris_types_analysis, ris_type_plot)
print("Done!")

Processing type: detail
Processing year: 2017
Opening areas shp: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\plot\fixed\risk\detail\2017\rd.shp
Opening mobilization: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\mob\fixed\2017.csv
Starting process parallel


## Importing process

## Generate outputs
In this section all csv files, which will be upload to database, are generated from inputs files.

In [3]:
# Getting parameters for processing data
ris_years = map(str.strip, conf.loc[conf["parameter"] == "ris_years","value"].values[0].split(","))
ris_types_analysis =  map(str.strip, conf.loc[conf["parameter"] == "ris_types_analysis","value"].values[0].split(","))
ris_type_plot = conf.loc[conf["parameter"] == "ris_type_plot","value"].values[0]

#impo.generate_outputs(inputs, outputs, ris_years, ris_types_analysis, ris_type_plot)
impo.generate_outputs(inputs, outputs, ["2017"], ris_types_analysis, ris_type_plot)
print("Done!")

Processing type: detail
Processing year: 2017
Opening areas shp: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\plot\fixed\total\detail\2017\total.shp
Extracting administrative levels
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\outputs\administrative_level.csv
Extracting administrative levels
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\outputs\localities.csv
Extracting administrative levels
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\outputs\cattle_rancher.csv
Starting analysis
Processing Cattle Rancher Risk
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\outputs\detail\2017\cattle_rancher_risk.csv
Processing mobilization
Opening mobilization: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\mob\fixed\2017.csv
Merging localities source
  id_source id_destination type_destination  HEMBRA BUFALINA DE 1 A 2 AÑO  \
0     34889        1165713           PREDIO                             0   
1     34889        1165718           PREDI

PermissionError: [Errno 13] Permission denied: 'D:\\CIAT\\Code\\BID\\dci_fauno\\data\\risk_calculator\\inputs\\mob\\log\\cattle_rancher_network.csv'